In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv("train_data.csv")
testa = pd.read_csv("testA_candidates.csv")

train_data.columns = ["userID", "itemID", "rating", "rel_days"]
testa.columns = ["userID", "itemID",]
testa["rating"] = 1

In [3]:
len(train_data), len(testa)

(529786, 36000)

In [4]:
train_data.head(20)

,userID,itemID,rating,rel_days
0,U00000,I02162,5,5
1,U00000,I00581,5,5
2,U00000,I02483,3,5
3,U00000,I00513,5,5
4,U00000,I00047,5,5
5,U00000,I01421,4,5
6,U00000,I00708,3,5
7,U00000,I02102,4,5
8,U00000,I01439,4,5
9,U00000,I00574,4,5


In [5]:
train_data["rating"].value_counts()

4    185194
3    139276
5    119264
2     56572
1     29480
Name: rating, dtype: int64

In [6]:
testa.head(20)

,userID,itemID,rating
0,U00000,I00168,1
1,U00000,I00689,1
2,U00000,I00728,1
3,U00000,I00997,1
4,U00000,I01117,1
5,U00000,I01603,1
6,U00000,I01781,1
7,U00000,I01838,1
8,U00000,I01853,1
9,U00000,I01904,1


In [7]:
from surprise import Dataset, NormalPredictor, Reader, SVD
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_data[["userID", "itemID", "rating"]], reader)

cross_validate(SVD(), data, cv=5)

{'test_rmse': array([0.88918527, 0.89051395, 0.88583408, 0.88651848, 0.89354687]),
 'test_mae': array([0.69827768, 0.70115951, 0.696209  , 0.6972111 , 0.70188244]),
 'fit_time': (1.3831779956817627,
  1.3898470401763916,
  1.5484390258789062,
  1.416487216949463,
  1.419098138809204),
 'test_time': (0.26430392265319824,
  0.29062390327453613,
  0.281843900680542,
  0.27144885063171387,
  0.2620878219604492)}

In [15]:
from surprise import Dataset, NormalPredictor, Reader, SVD
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_data.sort_values('rel_days', ascending=False).groupby('userID').head(50)[["userID", "itemID", "rating"]], reader)

cross_validate(SVD(), data, cv=5)

{'test_rmse': array([0.93543965, 0.94163379, 0.9364176 , 0.94534036, 0.94391782]),
 'test_mae': array([0.74079006, 0.74420478, 0.74073175, 0.74479474, 0.74421382]),
 'fit_time': (0.36594510078430176,
  0.37517619132995605,
  0.41222500801086426,
  0.4228329658508301,
  0.39021801948547363),
 'test_time': (0.04922890663146973,
  0.07873082160949707,
  0.0757758617401123,
  0.04637598991394043,
  0.07519292831420898)}

In [18]:
from surprise.model_selection import KFold

kf = KFold(n_splits=5)
algo = SVD()

test_pred = np.zeros(len(testa))

data = Dataset.load_from_df(train_data[["userID", "itemID", "rating"]], reader)
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testa.values)
    test_pred += np.array([x.est for x in predictions])

In [19]:
testa["rating"] = test_pred / 5

In [20]:
top5_per_user = (
    testa
    .sort_values('rating', ascending=False)  # First sort by rating in descending order
    .groupby('userID')
    .head(5)  # Get top 5 rows per userID
)
top5_per_user = top5_per_user.sort_values(by="userID")
top5_per_user['rating'] = top5_per_user['rating'].round().astype(int)
top5_per_user.columns = ["u_id", "i_id", "pred_rating"]

In [21]:
top5_per_user.to_csv("submit.csv", index=None)